## Stock Market Analysis

In this notebook, we will discover and explore data from the stock market, particularly some dead stocks. We will see how to use UpstoxApi by [Upstox](https://upstox.com/) to get stock historical data, for 1 min chart(open, high, low, close).

### Getting the Instrument details

* instrument_key	-->	The unique identifier used across Upstox APIs for instrument identification.
* exchange_token	-->	The numerical identifier issued by the exchange representing the instrument.
* tradingsymbol	    -->	Shows the trading symbol which could be a combination of symbol name, instrument, expiry date etc.
* name	            -->	Name of the company (for equity instruments).
* last_price	    -->	Last traded price.
* expiry	        -->	Expiry date (for derivatives). Data format is yyyy-MM-dd
* strike	        -->	Indicates the predetermined price at which an option can be bought or sold when it's exercised.
* tick_size	        -->	Measure of the minimum upward or downward movement in the price of an instrument.
* lot_size	        -->	Minimum size in which the stock futures or index futures can be traded.
* instrument_type	-->	Instrument type of a particular contract. Possible values: FUTSTK, OPTSTK, OPTIDX etc.
* option_type	    -->	Option type of the option contracts (applicable only for options contract). Possible values: CE, PE
* exchange	        -->	Exchange to which the order is associated.Possible values: NSE_EQ, NSE_FO, NCD_FO, NSE_INDEX, BSE_EQ, BSE_FO, BCD_FO, BSE_INDEX, MCX_FO, NSE_COM etc.

In [20]:
import gzip
import json

# Path to the compressed JSON file
file_path = "NSE.json.gz"

# Function to extract and read JSON data
def extract_json_data(file_path):
    with gzip.open(file_path, 'rt', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Extract data
data = extract_json_data(file_path)

# Function to filter by segment
def filter_by_segment(data, segment=None, name=None, instrument_type=None, 
                      instrument_key=None, exchange_token=None, trading_symbol=None):
    
    filtered_data = []
    for item in data:
        if (
            (segment is None or item.get("segment") == segment)
            and (name is None or item.get("name") == name)
            and (instrument_type is None or item.get("instrument_type") == instrument_type)
            and (instrument_key is None or item.get("instrument_key") == instrument_key)
            and (exchange_token is None or item.get("exchange_token") == exchange_token)
            and (trading_symbol is None or item.get("trading_symbol") == trading_symbol)
        ):
            filtered_data.append(item)

    
    return filtered_data

# Example: Filter for 'NCD_FO' segment
filteredTicker = filter_by_segment(data, "NSE_EQ", "LAKSHYA POWERTECH LIMITED", None, None, None, None)

# Print the first 5 records from the filtered data
print(json.dumps(filteredTicker, indent=3))

[
   {
      "segment": "NSE_EQ",
      "name": "LAKSHYA POWERTECH LIMITED",
      "exchange": "NSE",
      "isin": "INE0VZS01015",
      "instrument_type": "SM",
      "instrument_key": "NSE_EQ|INE0VZS01015",
      "lot_size": 800,
      "freeze_quantity": 100000.0,
      "exchange_token": "25860",
      "tick_size": 5.0,
      "trading_symbol": "LAKSHYA",
      "qty_multiplier": 1.0,
      "security_type": "SME"
   }
]


In [19]:
import requests
import threading
from flask import Flask, request, jsonify, redirect, session

# Flask App
app = Flask(__name__)
app.secret_key = "2356"

@app.route('/')
def login():
    return redirect("https://api.upstox.com/v2/login/authorization/dialog?response_type=code&client_id=3146de73-0e14-4bac-af8c-b248c718e43e&redirect_uri=http://127.0.0.1:5000/callback&state=xyz")

@app.route('/callback')
def callback():
    code = request.args.get("code")
    if not code:
        return "Authorization failed", 400

    payload = {
        "code": code,
        "client_id": "3146de73-0e14-4bac-af8c-b248c718e43e",
        "client_secret": "tpfd7qwdmd",
        "redirect_uri": "http://127.0.0.1:5000/callback",
        "grant_type": "authorization_code",
    }
    response = requests.post("https://api.upstox.com/v2/login/authorization/token", data=payload)

    try:
        data = response.json()
    except requests.exceptions.JSONDecodeError:
        return "Failed to decode response from Upstox", 500

    if "access_token" not in data:
        return jsonify({"error": data}), 400

    session["access_token"] = data["access_token"]
    return redirect("/profile")

@app.route('/profile')
def profile():
    if "access_token" not in session:
        return redirect("/")
    
    headers = {"Authorization": f"Bearer {session['access_token']}"}
    response = requests.get("https://api.upstox.com/v2/user/profile", headers=headers)

    try:
        return jsonify(response.json())
    except requests.exceptions.JSONDecodeError:
        return "Failed to decode profile response", 500

# Run Flask in a background thread (if inside Jupyter)
def run_flask():
    app.run(debug=True, use_reloader=False)

threading.Thread(target=run_flask).start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [ ]:
def make_request(method, url, header=None, params=None, data=None):
    response = None
    try:
        if method == 'GET':
            request = requests.get(url, headers=header, params=params)
        else:
            raise ValueError("Not Valid HTTP Method")
        
        if request.status_code == 200:
            return request.json()
        else:
            return response
    except request.exceptions.RequestException as e:
        print(f'Error : {e}')
        return None

In [ ]:
interval= "1minute"
from_date = "2021-09-30"
to_date = "2021-10-15"

url= f"https://api.upstox.com/historical/nse_eq?symbol=LAKSHYAPOWER&interval={interval}&from={from_date}&to={to_date}"
header = {
    'accept': 'application/json',
    'Authorization': f'Bearer {session["access_token"]}'
    }

response = make_request('GET', url, header=header)
print(response)

url= f"https://api.upstox.com/v2/historical-candle/intraday/{instrument_key}/{interval}"
header = {
    'accept': 'application/json',
    'Authorization': f'Bearer {session["access_token"]}'
    }
response = make_request('GET', url, header=header)
print(response)